# Search space basics

In [12]:
from ablator import ParallelConfig, ModelConfig, OptimizerConfig, SchedulerConfig, TrainConfig, Derived, configclass
from ablator.main.configs import SearchSpace
from ablator.main.state import ExperimentState

import torch.nn as nn
import torch

from pathlib import Path
import tempfile

class MyModelConfig(ModelConfig):
    inp_size: Derived[int]
    hidden_dim: Derived[int]
    activation: Derived[str]
    dropout: Derived[float]

@configclass
class CustomParallelConfig(ParallelConfig):
    model_config: MyModelConfig

class MyCustomModel(nn.Module):
    def __init__(self, config: MyModelConfig) -> None:
        super().__init__()
        self.linear = nn.Linear(config.inp_size, config.hidden_dim)
        self.dropout = nn.Dropout(config.dropout)
        if config.activation == "relu":
            self.activate = nn.ReLU()
        elif config.activation == "elu":
            self.activate = nn.ELU()

    def forward(self, x: torch.Tensor):
        out = self.linear(x)
        out = self.dropout(out)
        out = self.activate(out)

        return {"preds": out, "labels": out}, x.sum().abs()

optimizer_config = OptimizerConfig(name="sgd", arguments={"lr": 0.1})
scheduler_config = SchedulerConfig("plateau", arguments={})
# scheduler_config = SchedulerConfig(name="cycle", arguments={"max_lr": 0.5, "total_steps": 50})
train_config = TrainConfig(
    dataset="test",
    batch_size=128,
    epochs=2,
    optimizer_config=optimizer_config,
    scheduler_config=scheduler_config,
)


search_space = {
    "train_config.optimizer_config.arguments.lr": SearchSpace(
        value_range=[0, 1], value_type="float"
    ),
    "train_config.scheduler_config.name": SearchSpace(
        categorical_values=["plateau"], value_type="int"
    ),


    "model_config.inp_size": SearchSpace(
        value_range=[100, 500], value_type="int"
    ),
    "model_config.hidden_dim": SearchSpace(
        value_range=[100, 500], value_type="int"
    ),
    "model_config.activation": SearchSpace(
        categorical_values=["relu", "elu"], value_type="int"
    ),
    "model_config.dropout": SearchSpace(
        value_range=[0, 1], value_type="float"
    ),
}

config = CustomParallelConfig(
    train_config=train_config,
    model_config=MyModelConfig(),
    verbose="silent",
    device="cpu",
    amp=False,
    search_space=search_space,
    optim_metrics={"acc": "max"},
    total_trials=5,
    concurrent_trials=1,
    ignore_invalid_params=False,
    gpu_mb_per_experiment=0,
    cpus_per_experiment=0.1,
)

with tempfile.TemporaryDirectory() as fp:
    ExperimentState(Path(fp), config)

NotADirectoryError: [WinError 267] The directory name is invalid: 'C:\\Users\\hieuc\\AppData\\Local\\Temp\\tmpkpm9i71j\\9a5a_9991_optuna.db'